# Project 01  - CarPole in Open AI
## [Tutorial]
https://www.youtube.com/watch?v=cO5g5qLrLSo&t=804s

# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.0 --user
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [2]:
import gym
import random
import custom_cartpole

In [3]:
env = gym.make('CustomCartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
states

4

In [5]:
actions

2

In [6]:
episodes = 10
for episode in range (1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

env.close()

Episode:1 Score:16.0
Episode:2 Score:12.0
Episode:3 Score:25.0
Episode:4 Score:17.0
Episode:5 Score:41.0
Episode:6 Score:16.0
Episode:7 Score:14.0
Episode:8 Score:19.0
Episode:9 Score:45.0
Episode:10 Score:13.0


# 2. Create a Deep Learning Model with Keras

In [23]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [24]:
def build_model (states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [25]:
model = build_model(states, actions)

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3.Build Agent with Keras-RL

In [27]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [28]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [29]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 12:05 - reward: 1.0000

C:\ProgramData\Anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 64s 6ms/step - reward: 1.0000: - ETA: 0s - r
95 episodes - episode_reward: 103.853 [9.000, 200.000] - loss: 2.396 - mae: 19.769 - mean_q: 40.134

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 68s 7ms/step - reward: 1.0000
51 episodes - episode_reward: 198.451 [178.000, 200.000] - loss: 3.905 - mae: 39.559 - mean_q: 80.004

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 74s 7ms/step - reward: 1.0000
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 3.332 - mae: 41.042 - mean_q: 82.657

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 86s 9ms/step - reward: 1.0000
50 episodes - episode_reward: 199.420 [187.000, 200.000] - loss: 4.389 - mae: 39.767 - mean_q: 79.823

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 72s 7ms/step - reward: 1.0000
done, took 363.388 seconds


In [30]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 134.000, steps: 134
Episode 2: reward: 142.000, steps: 142
Episode 3: reward: 146.000, steps: 146
Episode 4: reward: 156.000, steps: 156
Episode 5: reward: 140.000, steps: 140
Episode 6: reward: 138.000, steps: 138
Episode 7: reward: 132.000, steps: 132
Episode 8: reward: 146.000, steps: 146
Episode 9: reward: 164.000, steps: 164
Episode 10: reward: 146.000, steps: 146
Episode 11: reward: 156.000, steps: 156
Episode 12: reward: 176.000, steps: 176
Episode 13: reward: 140.000, steps: 140
Episode 14: reward: 172.000, steps: 172
Episode 15: reward: 166.000, steps: 166
Episode 16: reward: 152.000, steps: 152
Episode 17: reward: 168.000, steps: 168
Episode 18: reward: 158.000, steps: 158
Episode 19: reward: 178.000, steps: 178
Episode 20: reward: 160.000, steps: 160
Episode 21: reward: 150.000, steps: 150
Episode 22: reward: 138.000, steps: 138
Episode 23: reward: 138.000, steps: 138
Episode 24: reward: 142.000, steps: 142
Episode 25: reward: 

In [31]:
_ = dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...
Episode 1: reward: 158.000, steps: 158
Episode 2: reward: 140.000, steps: 140
Episode 3: reward: 136.000, steps: 136
Episode 4: reward: 142.000, steps: 142
Episode 5: reward: 156.000, steps: 156


# 4. Reloading Agent from Memory

In [32]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [33]:
del model
del dqn
del env

In [34]:
env = gym.make('CustomCartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [35]:
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [36]:
dqn.load_weights('dqn_weights.h5f')

In [37]:
_ = dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...
Episode 1: reward: 146.000, steps: 146
Episode 2: reward: 172.000, steps: 172
Episode 3: reward: 160.000, steps: 160
Episode 4: reward: 144.000, steps: 144
Episode 5: reward: 150.000, steps: 150
